In [ ]:
import numpy as np # used for numeric calculations and arrays
from scipy.interpolate import griddata # used for interpolating onto a grid
import netCDF4 as nc # used to open NetCDF4 files
from netCDF4 import Dataset # used to open NetCDF4 files
import os # used to communicate with operating system
import re # used to search

# Output Longitude and Latitude (to interpolate onto)

In [2]:
folder = 'C:/Users/rubym/OneDrive - Australian National University/Honours/Sea Level Ice Outputs/Outputs (copied across)/9 Sept/'
longlat = np.loadtxt(f'{folder}longlat') # load in the longitude and latitude grid that I want my data to match

long = longlat[:, 0] # set longitude column
lat = longlat[:, 1] # set latitude column
longlat = np.array([long, lat]).T # transpose an array of long and lat

# Ice 6G 122Ka

In [3]:
folder_path = 'C:/Users/rubym/OneDrive - Australian National University/Honours/Python Drafts/Non-Interpolated Ice Histories/ICE-6G_C_122Ka' 

file_list = sorted(
    [f for f in os.listdir(folder_path) if f.startswith("ice_thickness_") and f.endswith(".grd")],
    key=lambda x: int(x.split('_')[-1].split('.')[0]) if x.split('_')[-1].split('.')[0].isdigit() else float('inf'),
    reverse=True)

# identify and sort all files in the folder that start with "ice_thickness_" and end with ".grd"

counter = 1
times_list = [] # store the time extracted from each file

output_folder_path = 'C:/Users/rubym/OneDrive - Australian National University/Honours/Python Drafts/Interpolated Ice Histories/ICE-6G-122ka'

for file_name in file_list: # loop through all ice thickness files in the folder
    file_path = os.path.join(folder_path, file_name) # create filepaths that match the file names in the folder
    
    ice_data = Dataset(file_path) # open the files (NetCDF4)

    time = int(file_name.split('_')[-1].split('.')[0]) # extracts the numeric portion of the filename used as a time index and store it in a list
    times_list.append(time)

    ice_lat = ice_data['lat'][:] # set latitude column
    ice_lon = ice_data['lon'][:] # set longitude column
    ice_thick = ice_data['stgit'][:] # set ice thickness column
    
    ice_lon = np.where(ice_lon > 180, ice_lon - 360, ice_lon) # convert longitude from [0, 360] range to [-180, 180] to match other data
    
    lon_grid, lat_grid = np.meshgrid(ice_lon, ice_lat) # create a 2D grid of existing data points 
    points = np.array([lon_grid.flatten(), lat_grid.flatten()]).T # flatten grid into 1D arrays
    values = ice_thick.flatten() # flatten ice thickness into 1D array
    grid_icethick_linear = griddata(points, values, longlat, method='linear') # linearly interpolate ice thickness points onto the long lat grid from above

    nan_indices = np.isnan(grid_icethick_linear) # look for NaN values
    if np.any(nan_indices): # if there are any NaN values
       grid_icethick_nearest = griddata(points, values, longlat[nan_indices], method='nearest') # interpolate those values using the nearest point
       grid_icethick_linear[nan_indices] = grid_icethick_nearest

    output_file_path = os.path.join(output_folder_path, f'ice{counter}') # save the nterpolated ice thickness data to a file with a time index
    np.savetxt(output_file_path, grid_icethick_linear)
  
    counter += 1 # increase the counter so that the next file is named differently

In [4]:
times_array = np.array(times_list) # turn the list of times into an array
times_array = -times_array # set the times to poisitive

times_file_path = os.path.join(output_folder_path, 'times') # save the times array
np.savetxt(times_file_path, times_array)

# Ice 6G 26Ka

In [5]:
# same method as above

folder_path = 'C:/Users/rubym/OneDrive - Australian National University/Honours/Python Drafts/Non-Interpolated Ice Histories/ICE-6G_C_26Ka'

def extract_number(filename):
    match = re.search(r'I6_C\.VM5a_1deg\.(\d+(\.\d+)?)', filename)
    if match:
         return float(match.group(1))

file_list = sorted(
    [f for f in os.listdir(folder_path) if f.startswith("I6_C.VM5a_1deg.") and f.endswith(".nc")],
    key=extract_number,
    reverse=True
)

counter = 1
times_list = []

output_folder_path = 'C:/Users/rubym/OneDrive - Australian National University/Honours/Python Drafts/Interpolated Ice Histories/ICE-6G-26ka'

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    
    ice_data = Dataset(file_path)

    time = file_name.split('_1deg.')[-1].split('.nc')[0]
    
    times_list.append(time)

    ice_lat = ice_data['lat'][:]
    ice_lon = ice_data['lon'][:]
    ice_thick = ice_data['stgit'][:]
    
    ice_lon = np.where(ice_lon > 180, ice_lon - 360, ice_lon)
    
    lon_grid, lat_grid = np.meshgrid(ice_lon, ice_lat)
    points = np.array([lon_grid.flatten(), lat_grid.flatten()]).T
    values = ice_thick.flatten()
    grid_icethick_linear = griddata(points, values, longlat, method='linear')

    nan_indices = np.isnan(grid_icethick_linear)
    if np.any(nan_indices):
      grid_icethick_nearest = griddata(points, values, longlat[nan_indices], method='nearest')
      grid_icethick_linear[nan_indices] = grid_icethick_nearest

    output_file_path = os.path.join(output_folder_path, f'ice{counter}')  # No extension
    np.savetxt(output_file_path, grid_icethick_linear)
  
    counter += 1

In [6]:
# same method as above

times_array = np.array(times_list, dtype=float)
times_array = -times_array * 1000

times_file_path = os.path.join(output_folder_path, 'times')
np.savetxt(times_file_path, times_array)

# Ice 7G 26Ka

In [7]:
# same method as above

folder_path = 'C:/Users/rubym/OneDrive - Australian National University/Honours/Python Drafts/Non-Interpolated Ice Histories/ICE-7G_NA_26Ka'

def extract_number(filename):
    match = re.search(r'I7G_NA\.VM7_1deg\.(\d+(\.\d+)?)', filename)
    if match:
         return float(match.group(1))

file_list = sorted(
    [f for f in os.listdir(folder_path) if f.startswith("I7G_NA.VM7_1deg.") and f.endswith(".nc")],
    key=extract_number,
    reverse=True
)

counter = 1
times_list = []

output_folder_path = 'C:/Users/rubym/OneDrive - Australian National University/Honours/Python Drafts/Interpolated Ice Histories/ICE-7G'

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    
    ice_data = Dataset(file_path)

    time = file_name.split('_1deg.')[-1].split('.nc')[0]
    
    times_list.append(time)

    ice_lat = ice_data['lat'][:]
    ice_lon = ice_data['lon'][:]
    ice_thick = ice_data['stgit'][:]
    
    ice_lon = np.where(ice_lon > 180, ice_lon - 360, ice_lon)
    
    lon_grid, lat_grid = np.meshgrid(ice_lon, ice_lat)
    points = np.array([lon_grid.flatten(), lat_grid.flatten()]).T
    values = ice_thick.flatten()
    grid_icethick_linear = griddata(points, values, longlat, method='linear')

    nan_indices = np.isnan(grid_icethick_linear)
    if np.any(nan_indices):
      grid_icethick_nearest = griddata(points, values, longlat[nan_indices], method='nearest')
      grid_icethick_linear[nan_indices] = grid_icethick_nearest

    output_file_path = os.path.join(output_folder_path, f'ice{counter}')  # No extension
    np.savetxt(output_file_path, grid_icethick_linear)
  
    counter += 1

In [8]:
# same method as above

times_array = np.array(times_list, dtype=float)
times_array = -times_array * 1000

times_file_path = os.path.join(output_folder_path, 'times')
np.savetxt(times_file_path, times_array)

# PaleoMIST

In [9]:
# same method as above

folder_path = 'C:/Users/rubym/OneDrive - Australian National University/Honours/Python Drafts/Non-Interpolated Ice Histories/PaleoMIST_80ka'

file_list = sorted(
    [f for f in os.listdir(folder_path) if f.startswith("ice_thickness_") and f.endswith(".grd")],
    key=lambda x: int(x.split('_')[-1].split('.')[0]) if x.split('_')[-1].split('.')[0].isdigit() else float('inf'),
    reverse=True)

counter = 1
times_list = []

output_folder_path = 'C:/Users/rubym/OneDrive - Australian National University/Honours/Python Drafts/Interpolated Ice Histories/PaleoMIST'

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    
    ice_data = Dataset(file_path)

    time = int(file_name.split('_')[-1].split('.')[0])
    times_list.append(time)

    ice_lat = ice_data['lat'][:]
    ice_lon = ice_data['lon'][:]
    ice_thick = ice_data['z'][:]
    
    ice_lon = np.where(ice_lon > 180, ice_lon - 360, ice_lon)
    
    lon_grid, lat_grid = np.meshgrid(ice_lon, ice_lat)
    points = np.array([lon_grid.flatten(), lat_grid.flatten()]).T
    values = ice_thick.flatten()
    grid_icethick_linear = griddata(points, values, longlat, method='linear')

    nan_indices = np.isnan(grid_icethick_linear)
    if np.any(nan_indices):
      grid_icethick_nearest = griddata(points, values, longlat[nan_indices], method='nearest')
      grid_icethick_linear[nan_indices] = grid_icethick_nearest

    output_file_path = os.path.join(output_folder_path, f'ice{counter}')  # No extension
    np.savetxt(output_file_path, grid_icethick_linear)
  
    counter += 1

In [10]:
# same method as above

times_array = np.array(times_list)
times_array = -times_array

times_file_path = os.path.join(output_folder_path, 'times')
np.savetxt(times_file_path, times_array)